In [ ]:
#pip install cvxopt

In [ ]:
# !pip install IPython
from IPython.display import display, HTML, Image

In [ ]:
!git clone https://github.com/KU-DIC/LG_time_series_day15.git #코랩 사용

# [시계열 이상치 탐지 3 실습]
# SVDD

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- 함수 ( ) 안에서 shift+tab: arguments description. shift+tab+tab은 길게 볼 수 있도록

# __1. Support Vector Data Description (SVDD)__

### Support Vector Data Description (SVDD)
>SVDD의 목표는 Data를 feature space로 mapping 한 뒤, normal data들의 영역을 포함할 수 있는 가장 작은 Hypersphere를 찾는 것 <br>
>학습: Find a hypersphere enclosing all the normal instances in a feature space <br>
> 테스트: 새로운 데이터에 대하여 학습된 원을 넘으면 이상치에 가까운 것으로 판정함

In [ ]:
Image('/content/LG_time_series_day15/image/intro3.png')

In [ ]:
import sys
sys.path.append('/content/LG_time_series_day15/src/')

from svdd import SVDD
from visualize import Visualization as draw
from data import PrepareData as load

''' 기본 모듈 및 시각화 모듈 '''
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import preprocessing

# <br>__1. Data: Iris Dataset__

### Step1. 데이터 불러오기

In [ ]:
iris = datasets.load_iris()
data =  preprocessing.StandardScaler().fit_transform(iris.data)
label = iris.target

p_data = data[label == 0, :]
n_data = data[label != 0, :]
#{0:"setosa", 1:"versicolor", 2:"virginica"}

""" 레이블을 (1, -1)로 변경해야 합 """
p_label = np.mat(np.ones(p_data.shape[0])).T
n_label = np.mat(-np.ones(n_data.shape[0])).T

### Step2. 데이터 Split

In [ ]:
p_x, p_xt, p_y, p_yt = train_test_split(p_data, p_label, test_size=0.3, random_state=1)
n_x, n_xt, n_y, n_yt = train_test_split(n_data, n_label, test_size=0.9, random_state=1)
        
trainData = np.vstack((p_x, n_x))
testData = np.vstack((p_xt, n_xt))
trainLabel = np.vstack((p_y, n_y))
testLabel = np.vstack((p_yt, n_yt))

---

### Step 1. Base SVDD

In [ ]:
# set SVDD parameters
parameters = {"positive penalty": 0.9,
              "negative penalty": 0.8,
              "kernel": {"type": 'gauss', "width": 1/24},
              "option": {"display": 'on'}}


# construct an SVDD model
svdd = SVDD(parameters)

# train SVDD model
svdd.train(trainData, trainLabel)

# test SVDD model
distance, accuracy = svdd.test(testData, testLabel)

# visualize the results
draw.testResult(svdd, distance)
draw.testROC(testLabel, distance)

### Step 2. kernel function에 따른 결과 비교

In [ ]:
Image('/content/LG_time_series_day15/image/intro4.png')

In [ ]:
# kernel list
kernelList = {"1": {"type": 'gauss', "width": 1/24},
              "2": {"type": 'linear', "offset": 0},
              "3": {"type": 'ploy', "degree": 2, "offset": 0},
              "4": {"type": 'tanh', "gamma": 1e-4, "offset": 0},
              "5": {"type": 'lapl', "width": 1/12}
              }

for i in range(len(kernelList)):

    # set SVDD parameters
    parameters = {"positive penalty": 0.9,
                  "negative penalty": 0.8,
                  "kernel": kernelList.get(str(i+1)),
                  "option": {"display": 'on'}}
    
    # construct an SVDD model
    svdd = SVDD(parameters)
    
    # train SVDD model
    svdd.train(trainData, trainLabel)
      
    # test SVDD model
    distance, accuracy = svdd.test(testData, testLabel)
    draw.testResult(svdd, distance)
    draw.testROC(testLabel, distance)

---

# <br>__2. Data: Example. SVDD with dataset1__

In [ ]:
# 데이터 불러오기
df= pd.read_csv('/content/LG_time_series_day15/data/dataset1.csv',header=None)

df.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","Class"]

data = df.iloc[:,:-1]
label = df.iloc[:,[-1]]

p_data = data[label['Class']==0]
n_data = data[label['Class'] != 0]

p_label = np.mat(np.ones(p_data.shape[0])).T
n_label = np.mat(-np.ones(n_data.shape[0])).T
        
p_x, p_xt, p_y, p_yt = train_test_split(p_data, p_label, test_size=0.3, random_state=1)
n_x, n_xt, n_y, n_yt = train_test_split(n_data, n_label, test_size=0.9, random_state=1)
        
trainData = np.vstack((p_x, n_x))
testData = np.vstack((p_xt, n_xt))
trainLabel = np.vstack((p_y, n_y))
testLabel = np.vstack((p_yt, n_yt))


In [ ]:
# set SVDD parameters
parameters = {"positive penalty": 0.9,
              "negative penalty": 0.8,
              "kernel": {"type": 'gauss', "width": 1/24},
              "option": {"display": 'on'}}


# construct an SVDD model
svdd = SVDD(parameters)

# train SVDD model
svdd.train(trainData, trainLabel)

# test SVDD model
distance, accuracy = svdd.test(testData, testLabel)

# visualize the results
draw.testResult(svdd, distance)
draw.testROC(testLabel, distance)

# <br>__3. Data: NASA Bearing Dataset__

In [ ]:
# 데이터 불러오기

data= pd.read_csv('/content/LG_time_series_day15/data/nasa_bearing_dataset.csv', index_col=0)
data.index = pd.to_datetime(data.index)
data.head()

In [ ]:
X_train = data[data['data_type'] == 'train'].iloc[:, :4]
y_train = data[data['data_type'] == 'train'].iloc[:, -2].values

X_test = data[data['data_type'] == 'test'].iloc[:, :4]
y_test = data[data['data_type'] == 'test'].iloc[:, -2].values

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

In [ ]:
# train 데이터를 기반으로 train/test 데이터에 대하여 standard scaling 적용 (평균 0, 분산 1) 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(X_train)

In [ ]:
X_train_scaled = pd.DataFrame(scaler.transform(X_train), 
                              columns=X_train.columns, 
                              index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), 
                             columns=X_test.columns, 
                             index=X_test.index)

In [ ]:
# y_train (0,1) -> train_label (1.-1)로 변경
train_label = []
for i in y_train:
    if i == 0:
        train_label.append(1)
    else:
        train_label.append(-1)
        
train_label = np.array(train_label).reshape(-1,1)

In [ ]:
# y_test (0,1) -> test_label (1.-1)로 변경
test_label = []
for i in y_test:
    if i == 0:
        test_label.append(1)
    else:
        test_label.append(-1)
        
test_label = np.array(test_label).reshape(-1,1)

In [ ]:
# set SVDD parameters
parameters = {"positive penalty": 0.9,
              "negative penalty": 0.8,
              "kernel": {"type": 'gauss', "width": 1/24},
              "option": {"display": 'on'}}


# construct an SVDD model
svdd = SVDD(parameters)

# train SVDD model
svdd.train(trainData, trainLabel)

# test SVDD model
distance, accuracy = svdd.test(testData, testLabel)

# visualize the results
draw.testResult(svdd, distance)
draw.testROC(testLabel, distance)

# <br>__실습 과제 참고용(Data Import)__

In [ ]:
# dataset2 적용

df= pd.read_csv('/content/LG_time_series_day15/data/dataset2.csv',header=None)
df.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","Class"]


data = df.iloc[:,:-1]
label = df.iloc[:,[-1]]


p_data = data[label['Class']==0]
n_data = data[label['Class'] != 0]

p_label = np.mat(np.ones(p_data.shape[0])).T
n_label = np.mat(-np.ones(n_data.shape[0])).T
        
p_x, p_xt, p_y, p_yt = train_test_split(p_data, p_label, test_size=0.3, random_state=1)
n_x, n_xt, n_y, n_yt = train_test_split(n_data, n_label, test_size=0.9, random_state=1)
        
trainData = np.vstack((p_x, n_x))
testData = np.vstack((p_xt, n_xt))
trainLabel = np.vstack((p_y, n_y))
testLabel = np.vstack((p_yt, n_yt))